In [16]:
import numpy as np
import os
from findmissing import *

Nicholass-MacBook-Air-5.local


In [17]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [18]:
getNumUniqueSystems('labels.tsv')

1693

In [20]:
a,b,c = getPrev('foundPlanets.txt')

In [23]:
len(a[0]), len(b[0]), len(c)

(1154, 428, 111)

In [ ]:
prevFound, prevSkipped, prevMissing = getPrev()

In [ ]:
s,c,ccd = a[0]

In [ ]:
ccd

In [ ]:
a[0].replace("'","").strip('\n)(').split(', ')

In [ ]:
with open('foundPlanets.txt','r') as f:
  a = f.readlines()

In [ ]:
f = [[1,2,3],[2,3,4]]

np.shape(f)

In [ ]:
for a,b in np.transpose(f):
  print(a,b)

In [ ]:
np.hstack((f,p))[1]

In [ ]:
found = [[],[]]
skipped = [[],[]]
missing = []
for b in a:
  status = b.split(' ')[0]
  tic = b.split(' ')[1]
  info = ' '.join(b.split(' ')[2:])
  
  if status == 'Found':
    found[0].append(tic)
    found[1].append(info)
  elif status == 'Skipped':
    skipped[0].append(tic)
    skipped[1].append(info)
  else:
    missing.append(tic)

In [ ]:
missing

In [ ]:
info

In [ ]:
a

In [ ]:
def findInRaw(tic):
  baseDir = '/pdo/qlp-data/'
  sectors = os.listdir(baseDir)
  sectors = [sector for sector in sectors if 'sector' in sector]

  cams = ['cam1','cam2','cam3','cam4']
  ccds = ['ccd1', 'ccd2', 'ccd3', 'ccd4']

  ticlist = np.loadtxt('missing.txt',str)
  found = []
  foundCount=0
  notfound = []

  for tic in tqdm(ticlist):
    foundtic=False
    for sector in sectors[::-1]:
      foundsector=False
      for cam in cams:
        for ccd in ccds:
          testpath = os.path.join(baseDir,sector,'ffi/',cam,ccd,'LC/',tic)
          if os.path.exists(testpath):
            found.append([pdo, testpath, tic])
            foundsector=True
            foundtic=True
            foundCount+=1
      if foundsector:
        break
    print(foundCount)
    if not foundtic:
      notfound.append(tic)

In [ ]:
def loadPlanetInfo(pathtolabels):
  raw = np.loadtxt(pathtolabels, dtype=str, skiprows=2,delimiter='\t')
  data = a[1:]

  TICID = data[:,0]
  source = data[:,16]
  sectors_strs = data[:,-5]
  
  sectors = []
  for sectors_str in sectors_strs:
    float_sectors = []
    if len(sectors_str.strip()) == 0:
      sectors.append(None)
      continue
    for sector in sectors_str.strip().split(','):
      try:
        if '"' in sector:
          try:
            float_sec = int(sector.strip()[1:])
          except ValueError:
            float_sec = int(sector.strip()[:-1])
        else:
          float_sec = int(sector.strip())
      except Exception as e:
        print(e)
      float_sectors.append(float_sec)
    float_sectors.sort()
    sectors.append(float_sectors[-1])
  return TICID, source, sectors

In [ ]:
tics,source, sectors = loadPlanetInfo('labels.tsv')

In [ ]:
np.sum(source=='qlp')

In [ ]:
header = a[0]


In [ ]:
data[0]

In [ ]:
header

In [ ]:
r.sort()
r[-1]

In [ ]:
r = [1,4,3,26,7,2]

In [ ]:
sectors

In [ ]:
sectordir = '../../Data/'

In [ ]:
master = []
for i in range(1,6):
  foundfile = f'foundpdo{i}.txt'
  with open(foundfile,'r') as f:
    lines = f.readlines()
    for line in lines:
      master.append(line)

In [ ]:
anfailcount = 0
othercount=0
spoccount=0
otherfails = []
tics = []
for line in master:
  _, p, tic = line.strip().split(',')
  chunks = p.split('/')
  sector = chunks[3]
  cam = chunks[5]
  ccd = chunks[6]
  
  if tic in tics:
    continue
  else:
    tics.append(tic)
  
  if 'spoc' in sector:
    spoccount+=1
    continue
  
  anfile = os.path.join(sectordir,sector,'astroNetOutFiles.txt')
  
  with open(anfile,'r') as anf:
    anlines = anf.readlines()

  anfail=True
  for each in anlines:
    if cam in each and ccd in each:
      othercount+=1
      otherfails.append([tic.strip(),each.strip()])
      anfail=False
      break
  if anfail:
    anfailcount+=1

print(f'looking at {len(tics)}, {anfailcount} failed for astronet reasons, {othercount} other')

In [ ]:
with open('failed.txt','w') as f:
  for each in otherfails:
    print(','.join(each),file=f)